# pyraffle

Simple raffle script compatible with Luma downloaded guest file.

Make sure to copy the downloaded guest file and name it `guests.csv` before running this notebook.

## Import the libraries and set configuration

In [ ]:
from time import sleep
from IPython.display import display, Markdown
import numpy as np
import pandas as pd
from tqdm.notebook import trange


POP_PICKED_WINNER = True
ONLY_CHECKED_IN = False

## Load the guests dataset and initialize winner list

In [ ]:
guests_df = pd.read_csv("guests.csv")
winners_df = pd.DataFrame()

## Clean the guest list and save it

In [ ]:
guests_df = guests_df[["name", "email", "checked_in_at"]]

if ONLY_CHECKED_IN:
    guests_df = guests_df.dropna(subset="checked_in_at")

cleaned_df = guests_df.drop_duplicates(subset="name", keep="first")

cleaned_df.loc[:, "name"] = cleaned_df["name"].str.upper()

cleaned_df = cleaned_df.sort_values(by=["name"])
cleaned_df.reset_index(drop=True, inplace=True)

cleaned_df.to_csv("cleaned_guests.csv", index=False)

## The actual raffle function

In [ ]:
def raffle(df, prize=""):

    # Dismisses the raffle if the guest list is empty
    if len(df) < 1:
        display(Markdown("![Astonaut saying no](img/no-astronaut.png)\n"
            "# **Guest list empty! 🚫🚫🚫**\nRaffle can't done with an empty guest list."))
        return df
    
    global winners_df
    number = -1

    # Shuffle the list
    df = df.sample(frac=1).reset_index(drop=True)

    # Continuos random number genation for 5 seconds
    for i in trange(50):
        number = np.random.randint(len(df))
        sleep(.1)

    # Retrieves the raffle winner
    winner = df.loc[[number]]

    # Display a cute bear and announce the raffle winner
    display(Markdown("![Cute winner bear](img/cute-winner-bear.png)\n"
        f"# **Congratulations {winner.loc[number]["name"].title()}! 🥳🥳🥳**\n"
        f"{ f"## You won **{prize}**! 🎉🎉🎉\n" if prize else "" }"
        f"<details><summary>Show Email</summary>Email: {winner.loc[number]["email"]}</details>"))

    # Add the prize column and its value if the prize value is not empty
    if prize:
        winner.insert(len(winner.loc[number]), "prize", [prize,], True)
    
    winners_df = pd.concat([winners_df, winner]).reset_index(drop=True)

    # Removes the raffle winner from the list if POP_PICKED_WINNER is set to True
    if POP_PICKED_WINNER:
        df = df.drop(number)

    # Sorts the guest list again
    df = df.sort_values(by=["name"])
    df.reset_index(drop=True, inplace=True)
    
    return df

## Example raffle
You can customize this to your liking. Replace the value of the `prize` variable to make it more specfic.

In [ ]:
prize = "Example"

cleaned_df = raffle(cleaned_df, prize)

## Save winners to CSV

In [ ]:
winners_df.to_csv("winners.csv", index=False)